# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the CSV file
csv_file = "../WeatherPy/output_data/cities.csv"
city_data = pd.read_csv(csv_file)
city_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kaitangata,-46.2817,169.8464,55.00,76,100,13.00,NZ,1619728647
1,1,rikitea,-23.1203,-134.9692,76.48,68,99,10.13,PF,1619728648
2,2,hilo,19.7297,-155.0900,80.60,57,20,3.44,US,1619728381
3,3,hermanus,-34.4187,19.2345,55.99,74,80,3.00,ZA,1619728648
4,4,ushuaia,-54.8000,-68.3000,42.80,56,20,5.75,AR,1619728650
...,...,...,...,...,...,...,...,...,...,...
548,548,naron,43.5167,-8.1528,54.00,76,40,5.75,ES,1619728929
549,549,san luis,-33.2950,-66.3356,70.12,40,0,9.17,AR,1619728930
550,550,salekhard,66.5300,66.6019,24.78,96,100,7.87,RU,1619728930
551,551,sar-e pul,35.8333,66.1667,64.36,35,14,6.33,AF,1619728931


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure GMaps
gmaps.configure(api_key=g_key)

# Set the data set
locations = city_data[["Lat", "Lng"]].astype(float)
humidity  = city_data["Humidity"].astype(float)

In [4]:
# Configure the map layout
fig = gmaps.figure(layout={
        'width': '800px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
}, zoom_level=2, center=(46.0, -5.0))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Search Criterias
filtered_city_data = city_data.loc[(city_data['Max Temp']<80) & 
                                   (city_data['Max Temp']>70) & 
                                   (city_data['Wind Speed']<10) & 
                                   (city_data['Cloudiness']==0),:]
filtered_city_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
269,269,port hedland,-20.3167,118.5667,71.60,60,0,6.91,AU,1619728784
296,296,santa maria,-29.6842,-53.8069,73.40,49,0,9.22,BR,1619728795
324,324,beloha,-25.1667,45.0500,74.07,69,0,7.45,MG,1619728810
463,463,karratha,-20.7377,116.8463,78.89,34,0,7.81,AU,1619728872
479,479,costa rica,-23.4167,-54.6500,71.80,41,0,3.96,BR,1619728878
507,507,sakakah,29.9697,40.2064,75.20,14,0,3.44,SA,1619728889
549,549,san luis,-33.2950,-66.3356,70.12,40,0,9.17,AR,1619728930


In [6]:
len(filtered_city_data)

7

In [7]:
# NaN values dropped if present
filtered_city_data = filtered_city_data.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels = []

#Loop to get the hotels using google maps
for index, row in filtered_city_data.iterrows():
    lat = filtered_city_data['Lat'][index]
    lng = filtered_city_data['Lng'][index]
    params = {
        "location": f"{lat},{lng}",
        "keyword": "hotel",
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }
    try:
        response = requests.get(base_url, params=params).json()
        
        hotels.append(response['results'][0]['name'])    
    except IndexError:
        hotels.append("")
        pass

In [10]:
# New Data Frame with the first Hotel found
hotel_df = filtered_city_data
hotel_df['Hotel Name'] = hotels
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
269,269,port hedland,-20.3167,118.5667,71.60,60,0,6.91,AU,1619728784,Hedland Hotel
296,296,santa maria,-29.6842,-53.8069,73.40,49,0,9.22,BR,1619728795,Altadomo Hotel
324,324,beloha,-25.1667,45.0500,74.07,69,0,7.45,MG,1619728810,
463,463,karratha,-20.7377,116.8463,78.89,34,0,7.81,AU,1619728872,Karratha International Hotel
479,479,costa rica,-23.4167,-54.6500,71.80,41,0,3.96,BR,1619728878,
507,507,sakakah,29.9697,40.2064,75.20,14,0,3.44,SA,1619728889,Raoum Inn Hotel
549,549,san luis,-33.2950,-66.3356,70.12,40,0,9.17,AR,1619728930,GRAN HOTEL SAN LUIS GOLD


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))